### **Libraries:**


---




In [0]:
import pandas as pd

### **Data setup**:


---

In [0]:
!mkdir data

In [0]:
!pip install kaggle

In [0]:
!mkdir .kaggle

In [0]:
import json
# token = {'username':'hemantkshirsagar8','key':''}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c mlsp-2014-mri -p /content/data

In [0]:
!unzip /content/data/\*.zip -d /content/data/

In [0]:
!rm -r /content/data/*.zip
# !rm -r /content/data/*.m
# !rm -r /content/data/*.R
# !rm -r /content/data/*.nii

## **About Data:**



---



**Train.zip:**

**train_labels.csv** - Labels for the training set. The labels are indicated in the "Class" column. 0 = 'Healthy Control', 1 = 'Schizophrenic Patient'

**train_FNC.csv** - FNC features for the training set. These are correlation values. They describe the connection level between pairs of brain maps over time.

**train_SBM.csv** - SBM features for the training set. These are standardized weights. They describe the expression level of ICA brain maps derived from gray-matter concentration.

**Test.zip:**

**test_FNC.csv** - FNC features for the test set. Test subject labels have been removed. Your task is to predict these unknown labels from the provided features.

**test_SBM.csv** - SBM features for the test set. Test subject labels have been removed.


**About FNC Features:**

Functional Network Connectivity (FNC) are correlation values that summarize the overall connection between independent brain maps over time. Therefore, the FNC feature gives a picture of the connectivity pattern over time between independent networks (or brain maps). The provided FNC information was obtained from functional magnetic resonance imaging (fMRI) from a set of schizophrenic patients and healthy controls at rest, using group independent component analysis (GICA). The GICA decomposition of the fMRI data resulted in a set of brain maps, and corresponding timecourses. 

These timecourses indicated the activity level of the corresponding brain map at each point in time. The FNC feature are the correlations between these timecourses. In a way, FNC indicates a subject's overall level of 'synchronicity' between brain areas. Because this information is derived from functional MRI scans, FNCs are considered a functional modality feature (i.e., they describe patterns of the brain function). More about FNCs can be found here: [FNC paper](http://cercor.oxfordjournals.org/content/early/2012/11/09/cercor.bhs352.abstract).

**About SBM Loadings:**

Source-Based Morphometry (SBM) loadings correspond to the weights of brain maps obtained from the application of independent component analysis (ICA) on the gray-matter concentration maps of all subjects. Gray-matter corresponds to the outer-sheet of the brain; it is the brain region in which much of the brain signal processing actually occurs. In a way, the concentration of gray-matter is indicative of the "computational power" available in a certain region of the brain. Processing gray-matter concentration maps with ICA yields independent brain maps whose expression levels (i.e., loadings) vary across subjects. 

Simply put, a near-zero loading for a given ICA-derived brain map indicates that the brain regions outlined in that map are lowly present in the subject (i.e., the gray-matter concentration in those regions are very low in that subject). Because this information is derived from structural MRI scans, SBM loadings are considered a structural modality feature (i.e., they describe patterns of the brain structure). More about SBM loadings can be found here: [SBM paper](http://www.ncbi.nlm.nih.gov/pubmed/22470337).

Further details about data can be found here, at [kaggle competition](https://www.kaggle.com/c/mlsp-2014-mri/data).
